### Lab2 (a) Model preperation

In [10]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [12]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\3205097103.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9150


### Lab2 (b) Prune by percentage

In [23]:
def prune_by_percentage(layer, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    layer_weight = layer.weight.data.cpu().numpy()
    # Compute the q-th percentile of the abs of the converted array
    threshold = np.percentile(np.abs(layer_weight), q)
    # Generate a binary mask same shape as weight to decide which element to prune
    bi_mask = np.abs(layer_weight) > threshold
    # Convert mask to torch tensor and put on GPU
    bi_mask = torch.tensor(bi_mask).cuda()
    # Multiply the weight by mask to perform pruning
    layer.weight.data = layer.weight.data * bi_mask
    

In [24]:
net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=20.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\1467084968.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Sparsity of head_conv.0.conv: 0.2013888888888889
Sparsity of body_op.0.conv1.0.conv: 0.20008680555555555
Sparsity of body_op.0.conv2.0.conv: 0.20008680555555555
Sparsity of body_op.1.conv1.0.conv: 0.20008680555555555
Sparsity of body_op.1.conv2.0.conv: 0.20008680555555555
Sparsity of body_op.2.conv1.0.conv: 0.20008680555555555
Sparsity of body_op.2.conv2.0.conv: 0.20008680555555555
Sparsity of body_op.3.conv1.0.conv: 0.20008680555555555
Sparsity of body_op.3.conv2.0.conv: 0.20008680555555555
Sparsity of body_op.4.conv1.0.conv: 0.20008680555555555
Sparsity of body_op.4.conv2.0.conv: 0.20008680555555555
Sparsity of body_op.5.conv1.0.conv: 0.20008680555555555
Sparsity of body_op.5.conv2.0.conv: 0.20008680555555555
Sparsity of body_op.6.conv1.0.conv: 0.20003255208333334
Sparsity of body_op.6.conv2.0.conv: 0.2000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.2000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.2000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.2000054253472222
Spa

### Lab2 (c) Finetune pruned model

In [14]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element 
            weight_mask[name] = 1 - (layer.weight.data == 0).float()
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data = layer.weight.data * weight_mask[name]
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()
    
    

In [15]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\93151597.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [28]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9717	acc=0.6835	198.3 examples/second
[Step=100]	Loss=0.8004	acc=0.7372	9249.6 examples/second
[Step=150]	Loss=0.7171	acc=0.7621	9144.4 examples/second
Test Loss=0.5536, Test acc=0.8183
Saving...

Epoch: 1
[Step=50]	Loss=0.4729	acc=0.8405	188.2 examples/second
[Step=100]	Loss=0.4553	acc=0.8468	9349.7 examples/second
[Step=150]	Loss=0.4464	acc=0.8495	9307.3 examples/second
Test Loss=0.4868, Test acc=0.8393
Saving...

Epoch: 2
[Step=50]	Loss=0.4189	acc=0.8561	201.2 examples/second
[Step=100]	Loss=0.4087	acc=0.8597	9449.2 examples/second
[Step=150]	Loss=0.3973	acc=0.8634	9097.7 examples/second
Test Loss=0.4561, Test acc=0.8494
Saving...

Epoch: 3
[Step=50]	Loss=0.3614	acc=0.8788	200.4 examples/second
[Step=100]	Loss=0.3605	acc=0.8772	9571.0 examples/second
[Step=150]	Loss=0.3606	acc=0.8770	9467.0 examples/second
Test Loss=0.4360, Test acc=0.8568
Saving...

Epoch: 4
[Step=50]	Loss=0.3433	acc=0.8829	203.5 examples/second
[Step=100]	Loss=0.3418	acc=0.8831	9492.1 ex

In [29]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

C:\Users\kojil\AppData\Local\Temp\ipykernel_29456\2640758429.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_finetune.pt"))


Files already downloaded and verified
Test Loss=0.3669, Test accuracy=0.8786


### Lab2 (d) Iterative pruning

In [30]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = 8*(epoch+1)
                print("Pruning "+name+" with q="+str(q))
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
Pruning head_conv.0.conv with q=8
Pruning body_op.0.conv1.0.conv with q=8
Pruning body_op.0.conv2.0.conv with q=8
Pruning body_op.1.conv1.0.conv with q=8
Pruning body_op.1.conv2.0.conv with q=8
Pruning body_op.2.conv1.0.conv with q=8
Pruning body_op.2.conv2.0.conv with q=8
Pruning body_op.3.conv1.0.conv with q=8
Pruning body_op.3.conv2.0.conv with q=8
Pruning body_op.4.conv1.0.conv with q=8
Pruning body_op.4.conv2.0.conv with q=8
Pruning body_op.5.conv1.0.conv with q=8
Pruning body_op.5.conv2.0.conv with q=8
Pruning body_op.6.conv1.0.conv with q=8
Pruning body_op.6.conv2.0.conv with q=8
Pruning body_op.7.conv1.0.conv with q=8
Pruning body_op.7.conv2.0.conv with q=8
Pruning body_op.8.conv1.0.conv with q=8
Pruning body_op.8.conv2.0.conv with q=8
Pruning final_fc.linear with q=8


C:\Users\kojil\AppData\Local\Temp\ipykernel_29456\1262236627.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


[Step=50]	Loss=0.0490	acc=0.9844	201.4 examples/second
[Step=100]	Loss=0.0486	acc=0.9848	8592.1 examples/second
[Step=150]	Loss=0.0479	acc=0.9848	8450.6 examples/second
Test Loss=0.3246, Test acc=0.9149

Epoch: 1
Pruning head_conv.0.conv with q=16
Pruning body_op.0.conv1.0.conv with q=16
Pruning body_op.0.conv2.0.conv with q=16
Pruning body_op.1.conv1.0.conv with q=16
Pruning body_op.1.conv2.0.conv with q=16
Pruning body_op.2.conv1.0.conv with q=16
Pruning body_op.2.conv2.0.conv with q=16
Pruning body_op.3.conv1.0.conv with q=16
Pruning body_op.3.conv2.0.conv with q=16
Pruning body_op.4.conv1.0.conv with q=16
Pruning body_op.4.conv2.0.conv with q=16
Pruning body_op.5.conv1.0.conv with q=16
Pruning body_op.5.conv2.0.conv with q=16
Pruning body_op.6.conv1.0.conv with q=16
Pruning body_op.6.conv2.0.conv with q=16
Pruning body_op.7.conv1.0.conv with q=16
Pruning body_op.7.conv2.0.conv with q=16
Pruning body_op.8.conv1.0.conv with q=16
Pruning body_op.8.conv2.0.conv with q=16
Pruning final_

In [31]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

C:\Users\kojil\AppData\Local\Temp\ipykernel_29456\2153638618.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_iterative_prune.pt

Files already downloaded and verified
Test Loss=0.3765, Test accuracy=0.8758


### Lab2 (e) Global iterative pruning

In [16]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            layer_weight = layer.weight.data.cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(layer_weight.flatten())
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.abs(flattened_weights), q)
    print("Global pruning threshold (q=%d): %.4f" % (q, thres))
    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            layer_weight = layer.weight.data.cpu().numpy()
            # Generate a binary mask same shape as weight to decide which element to prune
            bi_mask = np.abs(layer_weight) > thres
            # Convert mask to torch tensor and put on GPU
            bi_mask = torch.tensor(bi_mask).cuda()
            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data * bi_mask

In [17]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        # Increase model sparsity
        q = 8*(epoch+1)
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\1584474449.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))



Epoch: 0
Global pruning threshold (q=8): 0.0085
[Step=50]	Loss=0.0473	acc=0.9847	150.4 examples/second
[Step=100]	Loss=0.0469	acc=0.9853	5741.1 examples/second
[Step=150]	Loss=0.0470	acc=0.9850	5794.0 examples/second
Test Loss=0.3231, Test acc=0.9133

Epoch: 1
Global pruning threshold (q=16): 0.0172
[Step=50]	Loss=0.0471	acc=0.9855	163.6 examples/second
[Step=100]	Loss=0.0484	acc=0.9850	7635.4 examples/second
[Step=150]	Loss=0.0476	acc=0.9849	8275.9 examples/second
Test Loss=0.3260, Test acc=0.9142

Epoch: 2
Global pruning threshold (q=24): 0.0262
[Step=50]	Loss=0.0509	acc=0.9825	120.6 examples/second
[Step=100]	Loss=0.0479	acc=0.9841	1089.3 examples/second
[Step=150]	Loss=0.0493	acc=0.9836	3325.0 examples/second
Test Loss=0.3268, Test acc=0.9133

Epoch: 3
Global pruning threshold (q=32): 0.0356
[Step=50]	Loss=0.0538	acc=0.9827	145.8 examples/second
[Step=100]	Loss=0.0525	acc=0.9832	3278.2 examples/second
[Step=150]	Loss=0.0521	acc=0.9833	3250.7 examples/second
Test Loss=0.3255, Test 

In [18]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np.prod(np_weight.shape)
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.30787037037037035
Sparsity of body_op.0.conv1.0.conv: 0.6566840277777778
Sparsity of body_op.0.conv2.0.conv: 0.6388888888888888
Sparsity of body_op.1.conv1.0.conv: 0.6267361111111112
Sparsity of body_op.1.conv2.0.conv: 0.6475694444444444
Sparsity of body_op.2.conv1.0.conv: 0.6302083333333334
Sparsity of body_op.2.conv2.0.conv: 0.6692708333333334
Sparsity of body_op.3.conv1.0.conv: 0.6245659722222222
Sparsity of body_op.3.conv2.0.conv: 0.6881510416666666
Sparsity of body_op.4.conv1.0.conv: 0.7254774305555556
Sparsity of body_op.4.conv2.0.conv: 0.7826605902777778
Sparsity of body_op.5.conv1.0.conv: 0.7250434027777778
Sparsity of body_op.5.conv2.0.conv: 0.8131510416666666
Sparsity of body_op.6.conv1.0.conv: 0.7324761284722222
Sparsity of body_op.6.conv2.0.conv: 0.7646755642361112
Sparsity of body_op.7.conv1.0.conv: 0.7768012152777778
Sparsity of body_op.7.conv2.0.conv: 0.8259006076388888
Sparsity of body_op.8.conv1.0.conv: 0.8529730902777778
Sparsity of bod

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\273133103.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_global_iterative_pr

Files already downloaded and verified
Test Loss=0.3490, Test accuracy=0.8842


### Lab 3 (b) and (c): Fixed-point quantization

In [21]:
# Define quantized model and load weight
Nbits = 3 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\3293098641.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Files already downloaded and verified
Test Loss=0.4226, Test accuracy=0.8875


In [22]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0746	acc=0.9732	168.4 examples/second
[Step=100]	Loss=0.0743	acc=0.9745	7056.0 examples/second
[Step=150]	Loss=0.0755	acc=0.9740	6826.0 examples/second
Test Loss=0.3414, Test acc=0.9077
Saving...

Epoch: 1
[Step=200]	Loss=0.0698	acc=0.9727	154.9 examples/second
[Step=250]	Loss=0.0683	acc=0.9765	7355.3 examples/second
[Step=300]	Loss=0.0674	acc=0.9768	6766.0 examples/second
[Step=350]	Loss=0.0667	acc=0.9770	6943.5 examples/second
Test Loss=0.3361, Test acc=0.9107
Saving...

Epoch: 2
[Step=400]	Loss=0.0590	acc=0.9795	141.9 examples/second
[Step=450]	Loss=0.0638	acc=0.9783	5582.5 examples/second
[Step=500]	Loss=0.0642	acc=0.9778	5715.2 examples/second
[Step=550]	Loss=0.0657	acc=0.9774	5305.7 examples/second
Test Loss=0.3413, Test acc=0.9107

Epoch: 3
[Step=600]	Loss=0.0710	acc=0.9775	129.1 examples/second
[Step=650]	Loss=0.0679	acc=0.9776	5734.3 examples/second
[Step

C:\Users\kojil\AppData\Local\Temp\ipykernel_31736\4294047440.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("quantized_net_after_finetune

Files already downloaded and verified
Test Loss=0.3289, Test accuracy=0.9147


### Lab3 (d) Quantize pruned model

In [ ]:
# Define quantized model and load weight
Nbits = 3 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)

In [ ]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

### Lab4 (a) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [ ]:
# check the performance of symmetric quantization with 6, 5, 4, 3, 2 bits
# Define quantized model and load weight
Nbits = 6

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)